# Astropy I: Introduction

So far we have looked at some interesting features that Python and some of its packages have to offer. We had a glimpse of the power of NumPy and Matplotlib. These are perhaps very versatile packages and find use at a lot of places. 

However, the scope for astronomical data analysis is very vast and is only getting wider with time. Since Python is the  primary programming language that astronomers prefer, some went on to make packages relevant to astronomy. What if all such packages could be found together under one umbrella? Well, the **Astropy Project** is the answer.

[Astropy](https://www.astropy.org/) is an open-source and community-developed Python package, with the first public version released in [2013](https://www.aanda.org/articles/aa/full_html/2013/10/aa22068-13/aa22068-13.html).

The purpose of Astropy is to provide core astronomy-related functionality to the community, including support for domain-specific file formats such as flexible image transport system (FITS) files, common ASCII table formats, unit and physical quantity conversions, physical constants specific to astronomy, celestial coordinate and time transformations, world coordinate system (WCS) support, generalized containers for representing gridded as well as tabular data, and a framework for cosmological transformations and conversions.

To know more about Astropy, you can check out the latest documentation [here](https://docs.astropy.org/en/stable/).

To begin with, we shall look at the ease of reading and writing files using `astropy.table`

In [ ]:
from astropy.io import ascii

`astropy.io.ascii` offers many methods to read and write  a wide range of ASCII data table formats. You can find a list of all supported formats [here](https://docs.astropy.org/en/stable/io/ascii/). Let us look at the list of galaxies we used in a previous tutorial. 

In [ ]:
galaxies = ascii.read('galaxies.csv')

print(galaxies)

The `write` function allows us to write data onto a file of a desired format supported by `ascii`. Let us try writing the first 15 rows of `galaxies` to a new `.csv` file. This file shall get stored in the location of this notebook.

In [ ]:
ascii.write(galaxies[0:15],"galaxies_modified.csv", format='csv')

## Introduction to FITS files

The most popular file format used in astronomy, Flexible Image Transport System ([FITS](https://fits.gsfc.nasa.gov/fits_documentation.html)), is a digital file format meant for storage of data: formatted as multi-dimensional arrays (2D for the example we'll look at), or tables. Astropy provides the `astropy.io.fits` package to deal with FITS files efficiently and access information with ease.

Let us begin with first importing this package.

In [ ]:
from astropy.io import fits

Let us import some tools for plotting as well, since we would also like to look at the pretty picture hidden in the huge array of numbers.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from astropy.stats import sigma_clipped_stats

%matplotlib inline

Note that we have also imported `astropy.stats.sigma_clipped_stats`. We shall see why at a later point.

Let us now open the FITS file given.

In [ ]:
m3_fits = fits.open('Messier3.fits')
m3_fits.info()

We have now read the given FITS file into our variable. `m3_fits.info()` displays a summary of the FITS file in the form of a list of an [HDUList](https://docs.astropy.org/en/stable/io/fits/api/hdulists.html#astropy.io.fits.HDUList), details of which are not important here.

Since we have only one [HDU object](https://docs.astropy.org/en/stable/io/fits/api/hdus.html) (again, details not important), we shall index our variable as `m3_fits[0]` to access the data it contains.

In [ ]:
m3_header = m3_fits[0].header
m3_header

The header of a FITS file contains a lot of information about the when and where and how of the image. The image we are using was taken by the [GROWTH-India Telescope](https://sites.google.com/view/growthindia/) located at Hanle in Ladakh. The header tells us all the necessary details about the telescope, the camera, the target being observed, the observing conditions and so on.

We can read out individual keyword values from the header in the following ways.

In [ ]:
m3_header['DATE-OBS']

In [ ]:
m3_header[3]

It is also possible to update values in the header file if necessary.

In [ ]:
m3_header['DATE-OBS']='April 10th, 2020'

m3_header

The header keywords often have comments against them to give a short explanation.

In [ ]:
m3_header.comments['EXPOSURE']

Let us now turn to the most important part of a FITS file - the data it stores. In the case of our example, this data is a 2D array of photon counts detected by the [CCD camera](https://en.wikipedia.org/wiki/Charge-coupled_device#Use_in_astronomy) which we shall soon interpret as a beautiful image of Messier 3.

In [ ]:
m3_data = m3_fits[0].data

In [ ]:
print(m3_data.shape, m3_data.dtype.name)

We can check the size of the array as well as the type of data it stores. 

We can also view parts of the array and update it with new values (Although, why would one want to do that? )

In [ ]:
print(m3_data[2020:2025,2010:2015])

We have finally reached the point where we shall plot our Messier object. However, an image taken using a CCD camera often has some saturated pixels, often due to [cosmic rays](https://en.wikipedia.org/wiki/Cosmic_ray). To eliminate the outliers that result due to saturated pixels, we use the `sigma_clipped_stats` to compute stats for our data and use them in our plotting function.

In [ ]:
mean, median, std = sigma_clipped_stats(m3_data)

plt.figure(figsize=(10,10))

#set the scale of the image based on its statistics
plt.imshow(m3_data, vmin = median - 5*std, vmax = median + 5*std, cmap='gray')
plt.colorbar()

Note that we have used the value 'gray' for the `cmap` parameter. You can try out other colormaps given [here](https://matplotlib.org/tutorials/colors/colormaps.html).

We can also plot only the desired part of the image by appropriate indexing. For this image, since the Messier 3 is located at the centre of the image, we can make a plot of the central 2000x2000 part of the image.

In [ ]:
plt.figure(figsize=(10,10))

plt.imshow(m3_data[1000:3000,1000:3000], vmin = median - 5*std, vmax = median + 5*std, cmap='viridis')
plt.colorbar()

## CCD Cameras in Astronomy

The FITS image we have used above was taken by the GROWTH-India Telescope, which uses a CCD camera. The dimensions of this camera are 4108x4096, indicating the number of individual pixels that collect data i.e. photons. CCDs are particularly popular because of their linear response to light, meaning that the reading of each pixel is linearly proportional to the number of photons falling on this pixel. Hence, the array of numbers that is stored in a FITS file is basically a map of how many photons are incident where.

Ideally, the dark sky should give zero counts everywhere except where stars are present. However, we shall see in a later tutorial that there are numerous sources of imperfections that make the analysis challenging. For now, let us accept those imperfections and try quantifying them.

If you check the plots above and look at the regions which do not have any stars, you'll see that the color doesn't correspond to the zero value on the adjacent colorbar. This finite count which is more or less uniform everywhere is known as the background count. The sources for this count shall be discussed in a later tutorial.

### Your assignment today is ...

1. To find out a reasonable estimate for the background count in the given image.
2. Locate an isolated star and make it disappear into the background.